In [ ]:
import sqlite3
import pandas as pd

In [ ]:
cards = pd.read_csv('mtg_cards.csv')

In [ ]:
cards['holofoil'] = cards['holofoil'].fillna(0)
cards['number'] = cards['number'].str.upper()
cards.head()

In [ ]:
con = sqlite3.connect('mtg.db')

In [ ]:
cur = con.cursor()

_create_table = """
CREATE TABLE IF NOT EXISTS card_collection(
  id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
  set_shortname TEXT NOT NULL,
  collector_number TEXT NOT NULL,
  holofoil INTEGER NOT NULL
);
"""
cur.execute(_create_table)

_create_table = """
DELETE FROM card_collection;
"""
cur.execute(_create_table)

In [ ]:
_query = """
    INSERT INTO card_collection(
        set_shortname, collector_number, holofoil
    ) VALUES(
        :set, :number, :holofoil
    );
"""

cur.executemany(_query, cards.to_dict('records'))
con.commit()

In [ ]:
_query = """
    SELECT 
        s.name,
        s.set_shortname,
        s.set_name,
        s.collector_number,
        c.holofoil,
        CASE
          WHEN c.holofoil = 1
          THEN p.usd_foil
          ELSE p.usd
        END AS price
    FROM card_collection c
    JOIN scryfall s
      ON c.set_shortname = s.set_shortname
     AND c.collector_number = s.collector_number
    JOIN scryfall_prices p
      ON s.id = p.card_id
"""

card_prices = pd.read_sql_query(_query, con)

In [ ]:
card_prices.groupby('set_name').agg({'price': 'sum'}).sort_values(by='price', ascending=False)

In [ ]:
card_prices['price'].sum()